In [ ]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.3/48.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 73.1 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
import pandas as pd
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Step 1: Import the required libraries and modules

# Define the number of qubits and layers
num_qubits = 7
num_layers = 2

# Initialize PennyLane device and quantum simulator
dev = qml.device("default.qubit", wires=num_qubits)

# Step 2: Preprocess the dataset

# Load the dataset and extract features and labels
dataset = pd.read_csv('/content/seeds.csv')
dataset = dataset.dropna()
dataset

,Area,Perimeter,Compactness,Kernel.Length,Kernel.Width,Asymmetry.Coeff,Kernel.Groove,Type
0,15.26,14.84,0.8710,5.763,3.312,2.221,5.220,1
1,14.88,14.57,0.8811,5.554,3.333,1.018,4.956,1
2,14.29,14.09,0.9050,5.291,3.337,2.699,4.825,1
3,13.84,13.94,0.8955,5.324,3.379,2.259,4.805,1
4,16.14,14.99,0.9034,5.658,3.562,1.355,5.175,1
...,...,...,...,...,...,...,...,...
194,12.19,13.20,0.8783,5.137,2.981,3.631,4.870,3
195,11.23,12.88,0.8511,5.140,2.795,4.325,5.003,3
196,13.20,13.66,0.8883,5.236,3.232,8.315,5.056,3
197,11.84,13.21,0.8521,5.175,2.836,3.598,5.044,3


In [ ]:


#dataset = load_supermarket_dataset()  # Replace with your dataset loading code
features = dataset[['Area','Perimeter','Compactness','Kernel.Length','Kernel.Width','Asymmetry.Coeff','Kernel.Groove']].values
labels = dataset['Type'].values

# Split the dataset into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

# Scale the features
scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)
features_test_scaled = scaler.transform(features_test)

# Step 3: Encode the dataset using a quantum feature map

# Define the quantum circuit
@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(num_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

# Initialize random weights
weights = np.random.random((num_layers, num_qubits, 3))

# Define the quantum feature map
def quantum_embedding(features, weights):
    norm = np.linalg.norm(features)
    features /= norm
    return [qnode(features[i], weights) for i in range(len(features))]

# Encode the training and testing features using the quantum feature map
features_train_encoded = quantum_embedding(features_train_scaled, weights)
features_test_encoded = quantum_embedding(features_test_scaled, weights)

# Step 4: Implement the QKNN algorithm

# Define the distance metric
def distance(x, y):
    return np.sqrt(np.sum((x - y)**2))

# Define the QKNN classifier
def qknn_classify(features_train, labels_train, features_test, k):
    predictions = []
    for i in range(len(features_test)):
        distances = [distance(features_test[i], features_train[j]) for j in range(len(features_train))]
        nearest_indices = np.argsort(distances)[:k]
        nearest_labels = labels_train[nearest_indices]
        unique_labels, counts = np.unique(nearest_labels, return_counts=True)
        prediction = unique_labels[np.argmax(counts)]
        predictions.append(prediction)
    return predictions

# Classify the test features using QKNN
k = 3  # Number of nearest neighbors to consider
predictions = qknn_classify(features_train_encoded, labels_train, features_test_encoded, k)

# Step 5: Measure the accuracy of the algorithm

# Compute the accuracy
accuracy = accuracy_score(labels_test, predictions)
print("Accuracy:",accuracy)



Accuracy: 0.825


In [ ]:

from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(labels_test, predictions, average='weighted')
recall = recall_score(labels_test, predictions, average='weighted')
f1 = f1_score(labels_test, predictions, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.8132142857142858
Recall: 0.825
F1-score: 0.8123076923076923


In [ ]:
import pandas as pd
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Step 1: Import the required libraries and modules

# Define the number of qubits and layers
num_qubits = 7
num_layers = 2

# Initialize PennyLane device and quantum simulator
dev = qml.device("default.qubit", wires=num_qubits)

# Step 2: Preprocess the dataset

# Load the dataset and extract features and labels
dataset = pd.read_csv('/content/supermarket_sales - Sheet1.csv')
#dataset = pd.read_csv('/content/supermarket_sales.csv')
dataset = dataset.dropna()

dataset



,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,233-67-5758,C,Naypyitaw,Normal,Male,Health and beauty,40.35,1,2.0175,42.3675,1/29/2019,13:46,Ewallet,40.35,4.761905,2.0175,6.2
996,303-96-2227,B,Mandalay,Normal,Female,Home and lifestyle,97.38,10,48.6900,1022.4900,3/2/2019,17:16,Ewallet,973.80,4.761905,48.6900,4.4
997,727-02-1313,A,Yangon,Member,Male,Food and beverages,31.84,1,1.5920,33.4320,2/9/2019,13:22,Cash,31.84,4.761905,1.5920,7.7
998,347-56-2442,A,Yangon,Normal,Male,Home and lifestyle,65.82,1,3.2910,69.1110,2/22/2019,15:33,Cash,65.82,4.761905,3.2910,4.1


In [ ]:
#dataset = load_supermarket_dataset()  # Replace with your dataset loading code
features = dataset[['Unit price','Quantity','Tax 5%','Total','cogs','gross income','Rating']].values
labels = dataset['Gender'].values

# Split the dataset into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

# Scale the features
scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)
features_test_scaled = scaler.transform(features_test)

# Step 3: Encode the dataset using a quantum feature map

# Define the quantum circuit
@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(num_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

# Initialize random weights
weights = np.random.random((num_layers, num_qubits, 3))

# Define the quantum feature map
def quantum_embedding(features, weights):
    norm = np.linalg.norm(features)
    features /= norm
    return [qnode(features[i], weights) for i in range(len(features))]

# Encode the training and testing features using the quantum feature map
features_train_encoded = quantum_embedding(features_train_scaled, weights)
features_test_encoded = quantum_embedding(features_test_scaled, weights)

# Step 4: Implement the QKNN algorithm

# Define the distance metric
def distance(x, y):
    return np.sqrt(np.sum((x - y)**2))

# Define the QKNN classifier
def qknn_classify(features_train, labels_train, features_test, k):
    predictions = []
    for i in range(len(features_test)):
        distances = [distance(features_test[i], features_train[j]) for j in range(len(features_train))]
        nearest_indices = np.argsort(distances)[:k]
        nearest_labels = labels_train[nearest_indices]
        unique_labels, counts = np.unique(nearest_labels, return_counts=True)
        prediction = unique_labels[np.argmax(counts)]
        predictions.append(prediction)
    return predictions

# Classify the test features using QKNN
k = 3  # Number of nearest neighbors to consider
predictions = qknn_classify(features_train_encoded, labels_train, features_test_encoded, k)

# Step 5: Measure the accuracy of the algorithm

# Compute the accuracy
accuracy = accuracy_score(labels_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.495


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(labels_test, predictions, average='weighted')
recall = recall_score(labels_test, predictions, average='weighted')
f1 = f1_score(labels_test, predictions, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.49354388095961127
Recall: 0.495
F1-score: 0.4936668761785041


#end of Supermarket

In [ ]:
import pennylane as qml
import pandas as pd
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Step 1: Import the required libraries and modules

# Define the number of qubits and layers
num_qubits = 5
num_layers = 2

# Initialize PennyLane device and quantum simulator
dev = qml.device("default.qubit", wires=num_qubits)

# Step 2: Preprocess the dataset

# Load the dataset and extract features and labels
dataset = pd.read_csv('/content/Iris.csv')
dataset = dataset.dropna()
dataset

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [ ]:
#dataset = load_supermarket_dataset()  # Replace with your dataset loading code
features = dataset[['Id','SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']].values
labels = dataset['Species'].values

# Split the dataset into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

# Scale the features
scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)
features_test_scaled = scaler.transform(features_test)

# Step 3: Encode the dataset using a quantum feature map

# Define the quantum circuit
@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(num_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

# Initialize random weights
weights = np.random.random((num_layers, num_qubits, 3))

# Define the quantum feature map
def quantum_embedding(features, weights):
    norm = np.linalg.norm(features)
    features /= norm
    return [qnode(features[i], weights) for i in range(len(features))]

# Encode the training and testing features using the quantum feature map
features_train_encoded = quantum_embedding(features_train_scaled, weights)
features_test_encoded = quantum_embedding(features_test_scaled, weights)

# Step 4: Implement the QKNN algorithm

# Define the distance metric
def distance(x, y):
    return np.sqrt(np.sum((x - y)**2))

# Define the QKNN classifier
def qknn_classify(features_train, labels_train, features_test, k):
    predictions = []
    for i in range(len(features_test)):
        distances = [distance(features_test[i], features_train[j]) for j in range(len(features_train))]
        nearest_indices = np.argsort(distances)[:k]
        nearest_labels = labels_train[nearest_indices]
        unique_labels, counts = np.unique(nearest_labels, return_counts=True)
        prediction = unique_labels[np.argmax(counts)]
        predictions.append(prediction)
    return predictions

# Classify the test features using QKNN
k = 3  # Number of nearest neighbors to consider
predictions = qknn_classify(features_train_encoded, labels_train, features_test_encoded, k)

# Step 5: Measure the accuracy of the algorithm

# Compute the accuracy
accuracy = accuracy_score(labels_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8666666666666667


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(labels_test, predictions, average='weighted')
recall = recall_score(labels_test, predictions, average='weighted')
f1 = f1_score(labels_test, predictions, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.9022222222222223
Recall: 0.8666666666666667
F1-score: 0.8578754578754579


#end of Iris

In [ ]:
import pennylane as qml
import pandas as pd
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Step 1: Import the required libraries and modules

# Define the number of qubits and layers
num_qubits = 5
num_layers = 2

# Initialize PennyLane device and quantum simulator
dev = qml.device("default.qubit", wires=num_qubits)

# Step 2: Preprocess the dataset

# Load the dataset and extract features and labels
dataset = pd.read_csv('/content/Breast_cancer_data.csv')
dataset = dataset.dropna()
dataset

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,diagnosis
0,17.99,10.38,122.80,1001.0,0.11840,0
1,20.57,17.77,132.90,1326.0,0.08474,0
2,19.69,21.25,130.00,1203.0,0.10960,0
3,11.42,20.38,77.58,386.1,0.14250,0
4,20.29,14.34,135.10,1297.0,0.10030,0
...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0
565,20.13,28.25,131.20,1261.0,0.09780,0
566,16.60,28.08,108.30,858.1,0.08455,0
567,20.60,29.33,140.10,1265.0,0.11780,0


In [ ]:
#dataset = load_supermarket_dataset()  # Replace with your dataset loading code
features = dataset[['mean_radius','mean_texture','mean_perimeter','mean_area','mean_smoothness']].values
labels = dataset['diagnosis'].values

# Split the dataset into training and testing sets
features_train, features_test, labels_train, labels_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

# Scale the features
scaler = StandardScaler()
features_train_scaled = scaler.fit_transform(features_train)
features_test_scaled = scaler.transform(features_test)

# Step 3: Encode the dataset using a quantum feature map

# Define the quantum circuit
@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(num_qubits))
    qml.templates.StronglyEntanglingLayers(weights, wires=range(num_qubits))
    return qml.expval(qml.PauliZ(0))

# Initialize random weights
weights = np.random.random((num_layers, num_qubits, 3))

# Define the quantum feature map
def quantum_embedding(features, weights):
    norm = np.linalg.norm(features)
    features /= norm
    return [qnode(features[i], weights) for i in range(len(features))]

# Encode the training and testing features using the quantum feature map
features_train_encoded = quantum_embedding(features_train_scaled, weights)
features_test_encoded = quantum_embedding(features_test_scaled, weights)

# Step 4: Implement the QKNN algorithm

# Define the distance metric
def distance(x, y):
    return np.sqrt(np.sum((x - y)**2))

# Define the QKNN classifier
def qknn_classify(features_train, labels_train, features_test, k):
    predictions = []
    for i in range(len(features_test)):
        distances = [distance(features_test[i], features_train[j]) for j in range(len(features_train))]
        nearest_indices = np.argsort(distances)[:k]
        nearest_labels = labels_train[nearest_indices]
        unique_labels, counts = np.unique(nearest_labels, return_counts=True)
        prediction = unique_labels[np.argmax(counts)]
        predictions.append(prediction)
    return predictions

# Classify the test features using QKNN
k = 3  # Number of nearest neighbors to consider
predictions = qknn_classify(features_train_encoded, labels_train, features_test_encoded, k)

# Step 5: Measure the accuracy of the algorithm

# Compute the accuracy
accuracy = accuracy_score(labels_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.8333333333333334


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(labels_test, predictions, average='weighted')
recall = recall_score(labels_test, predictions, average='weighted')
f1 = f1_score(labels_test, predictions, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Precision: 0.8482216708023159
Recall: 0.8333333333333334
F1-score: 0.8355889724310778
